In [1]:
import numpy as np
import argparse
import random
import os
from scipy import misc
from skimage import draw
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense
import keras.backend as K
import pandas as pd
model = Sequential()
# 3x128x128
model.add(Conv2D(32, (3, 3), strides = (1,1), padding ="same", input_shape=(480, 640, 3), data_format = 'channels_last'))
model.add(Activation("relu"))
model.add(Dropout(0.0))
model.add(BatchNormalization())
# 32x128x128
'''model.add(Convolution2D(32, (3, 3), padding="same", data_format = 'channels_last'))
model.add(Activation("relu"))'''
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.0))
model.add(BatchNormalization())
 # 64x64x64
model.add(Convolution2D(64, (3, 3), padding="same", data_format = 'channels_last'))
model.add(Activation("relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.0))
model.add(BatchNormalization())
# 128x32x32
model.add(Convolution2D(64, (3, 3), padding="same", data_format = 'channels_last'))
model.add(Activation("relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.0))
model.add(BatchNormalization())
#128x16x16
'''model.add(Convolution2D(256, (3, 3), padding="same", data_format = 'channels_last'))
model.add(Activation("relu"))'''
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.0))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.0))
model.add(BatchNormalization())
model.add(Dense(4))

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 480, 640, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 480, 640, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 480, 640, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 480, 640, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 240, 320, 32)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 240, 320, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 240, 320, 32)      128       
__________

In [4]:
def iou_loss(y_true, y_pred):
    # iou loss for bounding box prediction
    # input must be as [x1, y1, x2 - x1, y2 - y1]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] + 1) * K.abs(K.transpose(y_true)[3] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] + 1) * K.abs(K.transpose(y_pred)[3] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2] + K.transpose(y_true)[0] , K.transpose(y_pred)[2] + K.transpose(y_pred)[0])
    overlap_3 = K.minimum(K.transpose(y_true)[3] + K.transpose(y_true)[1] , K.transpose(y_pred)[3] + K.transpose(y_pred)[1])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    # loss for the iou value
    iou_loss = -K.log(iou)

    return iou_loss

In [5]:
def iou_metric(y_true, y_pred):
    # iou as metric for bounding box regression
    # input must be as [x1, y1, x2-x1, y2-y1]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] + 1) * K.abs(K.transpose(y_true)[3] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] + 1) * K.abs(K.transpose(y_pred)[3] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2] + K.transpose(y_true)[0] , K.transpose(y_pred)[2] + K.transpose(y_pred)[0])
    overlap_3 = K.minimum(K.transpose(y_true)[3] + K.transpose(y_true)[1] , K.transpose(y_pred)[3] + K.transpose(y_pred)[1])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())
    return -iou   

In [6]:
print("Compiling...")
model.compile(loss='mean_squared_error', optimizer=RMSprop(), metrics=[iou_metric])

Compiling...


In [7]:
model.load_weights('model_new_regress_best_new_aug_noreg.weights')

In [8]:
submission = pd.DataFrame(columns=['image_name','x1','x2','y1','y2'])

In [9]:
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
test = pd.read_csv('test.csv')
for i in tqdm(test.iterrows()):
    image = np.expand_dims(np.array(Image.open('test/' + i[1]['image_name']), dtype=np.uint8) , axis = 0)
    pred = model.predict(image)
    final_coords = pred[0]
    final_coords[2] = final_coords[2] + final_coords[0]
    final_coords[3] = final_coords[3] + final_coords[1]
    if(final_coords[0] < 0):
        final_coords[0] = 0
    if(final_coords[1] < 0):
        final_coords[1] = 0
    if(final_coords[2] > 640):
        final_coords[2] = 640
    if(final_coords[3] > 480):
        final_coords[3] = 480
    submission = submission.append({'image_name':i[1]['image_name'],'x1':final_coords[0],'x2':final_coords[2],'y1':final_coords[1],'y2':final_coords[3]}, ignore_index=True)


24045it [08:52, 45.19it/s]


In [12]:
submission.to_csv('submit_new_aug_someshitattempt3.csv', index = False)